In [0]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm


  # graph it
        

In [0]:
DATADIR = '/content/drive/My Drive/ucmerced/UCMerced_LandUse/Images/'

CATEGORIES = [ 'agricultural',  'airplane',    'baseballdiamond', 'beach',   'buildings',          'chaparral',         'denseresidential',
                'forest',        'freeway',     'golfcourse',      'harbor',  'intersection',       'mediumresidential', 'mobilehomepark',
                'overpass',      'parkinglot',  'river',           'runway',  'sparseresidential',  'storagetanks',      'tenniscourt' ]


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
from PIL import Image #import Python Image Library
import numpy as np
import os
import glob
import re
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

Using TensorFlow backend.


In [0]:
path = os.path.abspath('.cnn.py')

In [0]:
path = re.sub('[a-zA-Z\s._]+$', '', path) #remove unintended file

In [0]:
dirs = os.listdir(path+'/drive/My Drive/ucmerced/UCMerced_LandUse/Images/')

In [0]:
img_rows = 256
img_cols = 256
num_class = 21
path = os.path.abspath('.cnn.py') #absolute path of program
path = re.sub('[a-zA-Z\s._]+$', '', path) #remove unintended file
x_train = []
y_train = []
x_test = []
y_test = []
dirs = os.listdir(path+'/drive/My Drive/ucmerced/UCMerced_LandUse/Images/')
dirs=dirs[:-1]
print(len(dirs))
'''

'''

NameError: ignored

In [0]:
label = 0
for i in dirs:
	n = 0
	count = 0
	for pic in glob.glob(path+'/drive/My Drive/ucmerced/UCMerced_LandUse/Images/'+i+'/*.tif'):
		im = Image.open(pic)
		im = np.array(im)
		if((im.shape[0]==256) and (im.shape[1] ==256) and count <90): #get only 90 data
			r = im[:,:,0]
			g = im[:,:,1]
			b = im[:,:,2]
			if(n<5): # 5 data in beginning set as test data
				x_test.append([r,g,b])
				y_test.append([label])
			else: #remaining data set as training data
				x_train.append([r,g,b])
				y_train.append([label])
			n = n + 1
			count = count + 1
		#print(count)
	label = label + 1

NameError: ignored

In [0]:
from skimage import feature
import numpy as np
 
class LocalBinaryPatterns:
  def __init__(self, numPoints, radius):
# store the number of points and radius
    self.numPoints = numPoints
    self.radius = radius
  def describe(self, image, eps=1e-7):
		# compute the Local Binary Pattern representation
		# of the image, and then use the LBP representation
		# to build the histogram of patterns
    lbp = feature.local_binary_pattern(image, self.numPoints,
    self.radius, method="uniform")
    return lbp 

In [0]:
desc = LocalBinaryPatterns(24, 8)

In [0]:
x_trainbck=x_train
x_testbck=x_test


In [0]:
x_train=np.array(x_train)
x_test=np.array(x_test)

In [0]:
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 3)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 3)

input_shape = (img_rows, img_cols, 3)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
y_train = keras.utils.to_categorical(y_train, 21)
y_test = keras.utils.to_categorical(y_test, 21)

In [0]:
x_train_lbp=[]


In [0]:
for i in x_train:
  gray = cv2.cvtColor(i, cv2.COLOR_BGR2GRAY)
  hist = desc.describe(gray)
  x_train_lbp.append(hist)

In [0]:
x_train_lbp=np.array(x_train_lbp)

In [0]:
x_train_lbp.shape

(1785, 256, 256)

In [0]:
x_test_lbp=[]

In [0]:
for i in x_test:
  gray = cv2.cvtColor(i, cv2.COLOR_BGR2GRAY)
  hist = desc.describe(gray)
  x_test_lbp.append(hist)

In [0]:
x_test_lbp=np.array(x_test_lbp)

In [0]:
x_test_lbp.shape

(105, 256, 256)

In [0]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [0]:
x_train_lbp = x_train_lbp.reshape(x_train_lbp.shape[0], img_rows, img_cols,1)
x_test_lbp = x_test_lbp.reshape(x_test_lbp.shape[0], img_rows, img_cols,1)

input_shape = (img_rows, img_cols,1)

In [0]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=x_train_lbp.shape[1:]))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(21, activation='softmax'))
model.compile(loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
model.fit(x_train_lbp, y_train, batch_size=50, nb_epoch=100, verbose=1, validation_data=(x_test_lbp, y_test))

Train on 1785 samples, validate on 105 samples
Epoch 1/100
1785/1785 [==============================] - 9s 5ms/sample - loss: 37.6961 - acc: 0.0431 - val_loss: 3.0426 - val_acc: 0.0190
Epoch 2/100
1785/1785 [==============================] - 5s 3ms/sample - loss: 3.0380 - acc: 0.0594 - val_loss: 3.0417 - val_acc: 0.0286
Epoch 3/100
1785/1785 [==============================] - 5s 3ms/sample - loss: 2.9356 - acc: 0.1014 - val_loss: 2.9330 - val_acc: 0.0762
Epoch 4/100
1785/1785 [==============================] - 5s 3ms/sample - loss: 2.4167 - acc: 0.2891 - val_loss: 2.9826 - val_acc: 0.1143
Epoch 5/100
1785/1785 [==============================] - 5s 3ms/sample - loss: 1.7640 - acc: 0.4924 - val_loss: 3.2176 - val_acc: 0.1238
Epoch 6/100
1785/1785 [==============================] - 5s 3ms/sample - loss: 1.1651 - acc: 0.6717 - val_loss: 3.8491 - val_acc: 0.1143
Epoch 7/100
1785/1785 [==============================] - 5s 3ms/sample - loss: 0.8270 - acc: 0.7810 - val_loss: 3.4904 - val_acc: 

In [0]:
loss, acc = model.evaluate(x_test, y_test, verbose=0)
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))


Testing loss: 3.565957782382057, acc: 0.47619047846112933



In [0]:
DATADIR = '/content/drive/My Drive/ucmerced/UCMerced_LandUse/Images/'

CATEGORIES = [ 'agricultural',  'airplane',    'baseballdiamond', 'beach',   'buildings',          'chaparral',         'denseresidential',
                'forest',        'freeway',     'golfcourse',      'harbor',  'intersection',       'mediumresidential', 'mobilehomepark',
                'overpass',      'parkinglot',  'river',           'runway',  'sparseresidential',  'storagetanks',      'tenniscourt' ]
